In [1]:
%%writefile ".env"

PROJECT_ID="feux_cq"

COMMUN_PATH = "C:/"
PATH_INFOCENTRE_APP = ${COMMUN_PATH}Informatique/SIG/Application/Jupyterhub/
PATH_ETUDE = ${COMMUN_PATH}Informatique/SIG/Donnees/Oeil/Traitement_Donnees/SURFACES_BRULEES_SENTINEL/2022/220502_PreparationFeu2020/
PROJECT_PATH =${COMMUN_PATH}Users/oriane.bruyere/Documents/Projet_feux/Script/Step1_detection_error/
DATA_CATALOG_DIR = ${PATH_INFOCENTRE_APP}projets/catalogFiles/ 
DATA_OUTPUT_DIR = ${PROJECT_PATH}output/
DATA_TEMP = N:/Secretariat/Etude/A_Activites/2023/I-1-3_Operer_suivi_incendies/LIVRABLES/temp/
SIG_DATA_PATH = ${COMMUN_PATH}Informatique/SIG/Donnees/
DB_USER="jfnguyenvansoc"
DB_PWD="oeil"
DB_HOST="172.20.12.13"
DB_PORT=5432

DB_WORKSPACE="oeil_traitement"
DB_REF="oeil_reference"
DB_EXT="data_externe"
DB_SCHEMA = "feux_cq"
DB_SCHEMA_REF = "feux"

Overwriting .env


## Get missing burned areas files from INSIGHT server

In [30]:
import logging
from dotenv import load_dotenv
from filecmp import dircmp
from os import listdir
import os
import filecmp
import geopandas as gpd
import pandas as pd
import fiona
from sqlalchemy import create_engine
load_dotenv()
import subprocess
## Create log file 
logging.basicConfig(filename='C:/Users/oriane.bruyere/Documents/Projet_feux/Script/Step1_detection_error/example.log', encoding='utf-8', level=logging.INFO, force=True)

##################################################
## list every files from INSIGHT server repository
##################################################
 
#### choix des variable pour aller récupérer les fichiers sur le server

month='01'
year=2023

%%bash
rep_loc="/home/oriane.bruyere/MyDocuments/Projet_feux"
year=2023
echo $year

# Serveur distant
server_user="oeil"
server_ip="192.168.13.43"
server_path="../../mnt/geo-storage/FEUXV2/RESULTS/"$year
file="FireDetection_SENTINEL2B_20230420-230233-196_L2A_T58KGA.gpkg"

sshpass -p '03il2023!?' scp $server_user@$server_ip:$server_path/$file $rep_loc

# Vérification du statut de la commande scp
if [ $? -eq 0 ]; then
  echo "Le fichier $file a été copié avec succès dans $rep_loc"
  else
    echo "Erreur lors de la copie du fichier $file depuis le serveur."
    fi


## Open gpkg and add data to POSTGIS

In [3]:
fichiers_geopackages = [f for f in os.listdir(os.getenv("DATA_TEMP")) if f.endswith('.gpkg')]
#print(fichiers_geopackages)

# Si moins de deux fichiers, il n'y a rien à fusionner

if len(fichiers_geopackages) < 2:
    print("Moins de deux fichiers GeoPackage dans le répertoire. Aucune fusion nécessaire.")
    for fichier_geopackage in fichiers_geopackages:
        chemin_geopackage = os.path.join(os.getenv("DATA_TEMP"), fichier_geopackage)
        gdf = gpd.read_file(chemin_geopackage)
        gdf['date']=[x[:-11] for x in gdf['date']]
        print(gdf)
else:
    # Lire chaque GeoPackage et concaténer les GeoDataFrames
    gdf = gpd.GeoDataFrame()
    for chemin_fichier in fichiers_geopackages:
        print(chemin_fichier)
        chemin_geopackage = os.path.join(os.getenv("DATA_TEMP"), chemin_fichier)
        couches = fiona.listlayers(chemin_geopackage)

        for couche in couches:
            geo_df_couches = gpd.read_file(chemin_geopackage, layer=couche, driver='GPKG')
            # Concatène les couches au GeoDataFrame global
            gdf = pd.concat([gdf, geo_df_couches], ignore_index=True, sort=False)
 
    gdf['date']=[x[:-11] for x in gdf['date']]
    gdf.sort_values(['date'], ascending=[True], inplace=True) ### ordre chronologique et par commune
    print("Données fusionnées à partir des GeoPackages :")
    print(gdf)

FireDetection_SENTINEL2X_AOUT_SEPT_OCT.gpkg
FireDetection_SENTINEL2X_AVRIL-MAI-JUIN-JUILLET_2023.gpkg
FireDetection_SENTINEL2_Decembre2023.gpkg
FireDetection_SENTINEL2_FEVRIER-2023.gpkg
FireDetection_SENTINEL2_JANVIER-2023.gpkg
FireDetection_SENTINEL2_MARS-2023.gpkg
FireDetection_SENTINEL2_Novembre2023.gpkg
Données fusionnées à partir des GeoPackages :
           date                                        nom       province  \
25880  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
25879  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
25881  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
25882  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
25883  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
...         ...                                        ...            ...   
24387  20231231  SENTINEL2A_20231231-230228-192_L2A_T58KGA   Province Sud   
24388  20231231  SENTINEL2A_2

## Convertir les surfaces detectees en type MULTIPOLYGONE

In [4]:
from shapely.geometry import MultiPolygon

# Vérifier si la géométrie est de type Polygon
if gdf['geometry'].geom_type[0] == 'Polygon':
    gdf['geometry'] = gdf['geometry'].apply(lambda poly: MultiPolygon([poly]))

    print(gdf)
else:
    print("La géométrie n'est pas de type Polygon.")

           date                                        nom       province  \
25880  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
25879  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
25881  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
25882  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
25883  20230101  SENTINEL2A_20230101-232202-326_L2A_T58KDC  Province Nord   
...         ...                                        ...            ...   
24387  20231231  SENTINEL2A_20231231-230228-192_L2A_T58KGA   Province Sud   
24388  20231231  SENTINEL2A_20231231-230228-192_L2A_T58KGA   Province Sud   
24389  20231231  SENTINEL2A_20231231-230228-192_L2A_T58KGA   Province Sud   
24382  20231231  SENTINEL2A_20231231-230228-192_L2A_T58KGA   Province Sud   
24390  20231231  SENTINEL2A_20231231-230228-192_L2A_T58KGA   Province Sud   

            commune   surface       x       y cuirasse  cuira_area bdtopo  

## Création de la base de donnée dans POSTGIS

In [5]:
def ClePrimaire(curseur, schema, table, colonne):
    sql = " ALTER TABLE " + schema + "." + table  + " ADD CONSTRAINT " + table + "_" + colonne + "_seq PRIMARY KEY(" + colonne + ");"
    curseur.execute(sql)

In [6]:
conex = create_engine(f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PWD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_WORKSPACE")}')
gdf.to_postgis("surfaces_brulees_brute_control", conex,schema=os.getenv("DB_SCHEMA"),if_exists='replace')